#### ingestlap times folder

- read folders
- rename columns and add new columns
- drop unwanted columns
- write output to silver container

In [0]:
dbutils.widgets.text("data_source","testing")
value_data_source = dbutils.widgets.get("data_source")

In [0]:
%run "../../constants/configuration"

In [0]:
%run "../../utils/common_functions"

In [0]:
from pyspark.sql.functions import current_timestamp, col, concat, lit
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType

In [0]:
lap_times_schema = StructType([
    StructField("raceId", IntegerType(), False),
    StructField("driverId", IntegerType(), True),
    StructField("lap", IntegerType(), True),
    StructField("position", IntegerType(), True),
    StructField("time", StringType(), True),
    StructField("milliseconds", IntegerType(), True),
])

In [0]:
lap_times_df = spark.read \
    .schema(lap_times_schema)\
    .csv(f"{bronze_container_path}/lap_times")

In [0]:
lap_times_with_ingestion_date_df = add_ingestion_date(lap_times_df)

##### rename columns and add columns

In [0]:
lap_times_final_df = lap_times_with_ingestion_date_df.withColumnRenamed("driverId", "driver_id")\
                                 .withColumnRenamed("raceId", "race_id")\
                                 .withColumn("data_source", lit(value_data_source))


In [0]:
merge_condition = "tgt.race_id = src.race_id AND tgt.driver_id = src.driver_id AND tgt.lap = src.lap AND tgt.race_id = src.race_id"
merge_delta_data(lap_times_final_df, 'silver','lap_times', merge_condition, 'race_id')